<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/09_17_22_gossipcop_pytorch_biggraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install jsonlines

In [2]:
!git clone -qb working https://github.com/arampacha/PyTorch-BigGraph

In [3]:
%cd PyTorch-BigGraph/
!pip -qq install .
%cd /content/

/content/PyTorch-BigGraph
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
/content


In [5]:
from google.colab import drive
drive.mount('/gdrive')

from pathlib import Path
base_dir = Path("/gdrive/MyDrive/ResearchFND")
assert base_dir.exists()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [8]:
dataset_id = 'gossipcop'

In [9]:
(base_dir/f'{dataset_id}_edges.txt').exists()

True

## Data prep

> THIS STEP CAN BE SKIPPED IF `politifact_edges.txt` exists

In [ ]:
import pandas as pd
import ast
import json

In [ ]:
df = pd.read_csv(base_dir/f'{dataset_id}_agg.csv', index_col=0)
df.head(2)

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0


In [ ]:
df['tweets'] = df.tweets.map(ast.literal_eval)

In [ ]:
users_tweeted = df.tweets.map(lambda x: [int(e['user_id']) for e in x])

In [ ]:
len(users_tweeted), sum(users_tweeted.map(len) > 0)

(894, 149)

In [ ]:
import os
import jsonlines
import numpy as np
import torch

import IPython.display as ipd

In [ ]:
follow_src = []
follow_dst = []
with jsonlines.open(base_dir/"followers.jsonl") as reader:
    for line in reader:
        v = line["user_id"]
        for u in line["followers"]:
            follow_src.append(u)
            follow_dst.append(v)

In [ ]:
with jsonlines.open(base_dir/"following.jsonl") as reader:
    for line in reader:
        u = line["user_id"]
        for v in line["following"]:
            follow_src.append(u)
            follow_dst.append(v)

In [ ]:
with open(base_dir/'t2u.json') as f:
    t2u = json.load(f)

with open(base_dir/'users_info.json') as f:
    users_info = json.load(f)

In [ ]:
for u, info in users_info.items():
    for v in info['followers']:
        break
        follow_src.append(v)
        follow_dst.append(int(u))
    for v in info['friends']:
        follow_src.append(int(u))
        follow_dst.append(v)

In [ ]:
tweet_src = []
tweet_dst = []

for v, l in users_tweeted.iteritems():
    if not len(l):
        continue
    for u in l:
        tweet_src.append(u)
        tweet_dst.append(v)

In [ ]:
users_retweeted = df.retweets.map(lambda x: [t2u[str(e)] for e in x if (str(e) in t2u)])

In [ ]:
for v, l in users_retweeted.iteritems():
    if not len(l):
        continue
    for u in l:    
        tweet_src.append(u)
        tweet_dst.append(v)

In [ ]:
with open(base_dir/f'{dataset_id}_edges.txt', 'w') as f:
    for src, dst in zip(follow_src, follow_dst):
        f.write(f"{src}\t{dst}\tfollows\n")
    for src, dst in zip(tweet_src, tweet_dst):
        f.write(f"{src}\t{dst}\ttwitted\n")

## Training

In [10]:
import random
from pathlib import Path

import attr
import pkg_resources
from torchbiggraph.config import add_to_sys_path, ConfigFileLoader
from torchbiggraph.converters.importers import convert_input_data, TSVEdgelistReader
from torchbiggraph.converters.utils import download_url, extract_gzip, extract_tar
from torchbiggraph.eval import do_eval
from torchbiggraph.train import train
from torchbiggraph.util import (
    set_logging_verbosity,
    setup_logging,
    SubprocessInitializer,
)

In [11]:
data_dir = Path('./data')
data_dir.mkdir(parents=True, exist_ok=True)
fpath = base_dir/f'{dataset_id}_edges.txt'

In [12]:
import random

def split_edges(input_file, pct=0.8, train_file='data/train_edges.txt', valid_file='data/valid_edges.txt'):

    with open(input_file) as f:
        lines = f.readlines()

    follow_edges, tweet_edges = [], []
    for line in lines:
        if line.strip().endswith('follows'):
            follow_edges.append(line)
        else:
            tweet_edges.append(line)

    random.shuffle(follow_edges)
    random.shuffle(tweet_edges)
    follow_split, tweet_split = int(pct*len(follow_edges)), int(pct*len(tweet_edges))
    train_edges = follow_edges[:follow_split] + tweet_edges[:tweet_split]
    valid_edges = follow_edges[follow_split:] + tweet_edges[tweet_split:]

    with open(train_file, 'w') as f:
        f.writelines(train_edges)

    with open(valid_file, 'w') as f:
        f.writelines(valid_edges)

In [15]:
split_edges(fpath)

In [16]:
dimension = 64

loader = ConfigFileLoader()
config = loader.load_config('PyTorch-BigGraph/torchbiggraph/examples/configs/politifact_config.py', [[f'dimension={dimension}']])
set_logging_verbosity(0)
subprocess_init = SubprocessInitializer()
subprocess_init.register(setup_logging, 0)
subprocess_init.register(add_to_sys_path, loader.config_dir.name)
input_edge_paths = [data_dir/'train_edges.txt', data_dir/'valid_edges.txt']
output_train_path, output_test_path = config.edge_paths

In [17]:
convert_input_data(
    config.entities,
    config.relations,
    config.entity_path,
    config.edge_paths,
    input_edge_paths,
    TSVEdgelistReader(lhs_col=0, rhs_col=1, rel_col=2),
    dynamic_relations=config.dynamic_relations,
)

[2022-09-17 10:52:10.749828] Using the 2 relation types given in the config
[2022-09-17 10:52:10.755335] Searching for the entities in the edge files...
[2022-09-17 10:52:11.275445] Entity type user:
[2022-09-17 10:52:11.282003] - Found 83652 entities
[2022-09-17 10:52:11.288735] - Removing the ones with fewer than 1 occurrences...
[2022-09-17 10:52:11.324504] - Left with 83652 entities
[2022-09-17 10:52:11.326608] - Shuffling them...
[2022-09-17 10:52:11.437056] Entity type article:
[2022-09-17 10:52:11.438734] - Found 2117 entities
[2022-09-17 10:52:11.443935] - Removing the ones with fewer than 1 occurrences...
[2022-09-17 10:52:11.448384] - Left with 2117 entities
[2022-09-17 10:52:11.449560] - Shuffling them...
[2022-09-17 10:52:11.456033] Preparing counts and dictionaries for entities and relation types:
[2022-09-17 10:52:11.458138] - Writing count of entity type user and partition 0
[2022-09-17 10:52:11.532684] - Writing count of entity type article and partition 0
[2022-09-17 1

In [18]:
train_config = attr.evolve(config, edge_paths=[output_train_path])
train(train_config, subprocess_init=subprocess_init)

INFO:torchbiggraph:Loading entity counts...
INFO:torchbiggraph:Creating workers...
INFO:torchbiggraph:Initializing global model...
INFO:torchbiggraph:Starting epoch 1 / 30, edge path 1 / 1, edge chunk 1 / 1
INFO:torchbiggraph:Edge path: data/train_partitioned
INFO:torchbiggraph:still in queue: 0
INFO:torchbiggraph:Swapping partitioned embeddings None ( 0 , 0 )
INFO:torchbiggraph:Loading partitioned embeddings from checkpoint
INFO:torchbiggraph:( 0 , 0 ): Stats before training: loss:  540.151 , pos_rank:  1003.11 , mrr:  0.00380424 , r1:  0.000383926 , r10:  0.00479908 , r50:  0.0223317 , auc:  0.483555 , count:  7814
INFO:torchbiggraph:( 0 , 0 ): Training stats: loss:  19.1803 , reg:  0 , violators_lhs:  33.9436 , violators_rhs:  35.5767 , count:  148476
INFO:torchbiggraph:( 0 , 0 ): Stats after training: loss:  275.058 , pos_rank:  498.813 , mrr:  0.0692515 , r1:  0.0412081 , r10:  0.12452 , r50:  0.209368 , auc:  0.75979 , count:  7814
INFO:torchbiggraph:( 0 , 0 ): bucket 1 / 1 : Tra

In [19]:
eval_config = attr.evolve(config, edge_paths=[output_test_path])
do_eval(eval_config, subprocess_init=subprocess_init)

INFO:torchbiggraph:Starting edge path 1 / 1 (data/test_partitioned)
INFO:torchbiggraph:( 0 , 0 ): Processed 39073 edges in 1.9 s (0.02M/sec); load time: 0.069 s
INFO:torchbiggraph:Stats for edge path 1 / 1, bucket ( 0 , 0 ): loss:  14.1844 , pos_rank:  26.6417 , mrr:  0.21959 , r1:  0.133135 , r10:  0.406508 , r50:  0.801333 , auc:  0.755087 , count:  39073
INFO:torchbiggraph:
INFO:torchbiggraph:Stats for edge path 1 / 1: loss:  14.1844 , pos_rank:  26.6417 , mrr:  0.21959 , r1:  0.133135 , r10:  0.406508 , r50:  0.801333 , auc:  0.755087 , count:  39073
INFO:torchbiggraph:
INFO:torchbiggraph:
INFO:torchbiggraph:Stats: loss:  14.1844 , pos_rank:  26.6417 , mrr:  0.21959 , r1:  0.133135 , r10:  0.406508 , r50:  0.801333 , auc:  0.755087 , count:  39073
INFO:torchbiggraph:


## Save embeddings

In [ ]:
!torchbiggraph_export_to_tsv \
    'PyTorch-BigGraph/torchbiggraph/examples/configs/politifact_config.py' \
    --entities-output entity_embeddings.tsv \
    --relation-types-output relation_types_parameters.tsv

Loading relation types and entities...
Initializing model...
Loading model check point...
Writing entity embeddings...
Reading embeddings for entity type user partition 0 from checkpoint...
Writing embeddings for entity type user partition 0 to output file...
- Processed 5000/71759 entities so far...
- Processed 10000/71759 entities so far...
- Processed 15000/71759 entities so far...
- Processed 20000/71759 entities so far...
- Processed 25000/71759 entities so far...
- Processed 30000/71759 entities so far...
- Processed 35000/71759 entities so far...
- Processed 40000/71759 entities so far...
- Processed 45000/71759 entities so far...
- Processed 50000/71759 entities so far...
- Processed 55000/71759 entities so far...
- Processed 60000/71759 entities so far...
- Processed 65000/71759 entities so far...
- Processed 70000/71759 entities so far...
- Processed all 71759 entities
Reading embeddings for entity type article partition 0 from checkpoint...
Writing embeddings for entity type

### Save article embeddings

In [ ]:
import numpy as np

from torchbiggraph.checkpoint_manager import CheckpointManager
from torchbiggraph.config import ConfigFileLoader, ConfigSchema
from torchbiggraph.graph_storages import (
    AbstractEntityStorage,
    AbstractRelationTypeStorage,
    ENTITY_STORAGES,
    RELATION_TYPE_STORAGES,
)
from torchbiggraph.model import make_model, MultiRelationEmbedder

In [ ]:
ent_t_name = 'article'

In [ ]:
entity_storage = ENTITY_STORAGES.make_instance(config.entity_path)
relation_type_storage = RELATION_TYPE_STORAGES.make_instance(config.entity_path)

print("Initializing model...")
model = make_model(config)

print("Loading model check point...")
checkpoint_manager = CheckpointManager(config.checkpoint_path)
state_dict, _ = checkpoint_manager.read_model()
if state_dict is not None:
    model.load_state_dict(state_dict, strict=False)

Initializing model...
Loading model check point...


In [ ]:
ent_t_config = model.entities[ent_t_name]

In [ ]:
entities = []
embeddings = []
for partition in range(ent_t_config.num_partitions):
    print(
        f"Reading embeddings for entity type {ent_t_name} partition "
        f"{partition} from checkpoint..."
    )
    entities.extend(entity_storage.load_names(ent_t_name, partition))
    embeddings_, _ = checkpoint_manager.read(ent_t_name, partition)

    if model.global_embs is not None:
        embeddings_ += model.global_embs[model.EMB_PREFIX + ent_t_name]
    embeddings.append(embeddings_)

embeddings = np.concatenate(embeddings)

Reading embeddings for entity type article partition 0 from checkpoint...


In [ ]:
np.save(base_dir/f'{dataset_id}_pt_biggraph_article_idx_{dimension}.npy', np.array([int(e) for e in entities]))
np.save(base_dir/f'{dataset_id}_pt_biggraph_article_embeddings_{dimension}.npy', embeddings)